In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dspy

In [3]:
# Clear DSPy's global disk cache
if hasattr(dspy, 'cache') and hasattr(dspy.cache, 'disk_cache'):
    dspy.cache.disk_cache.clear()
    print("clear")

clear


In [4]:
import httpx
from dspy.teleprompt import GEPA

In [5]:
train = [
    dspy.Example(sentence="I love this!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="This is terrible.", emotion="sad").with_inputs("sentence"),
    dspy.Example(sentence="The weather is okay.", emotion="neutral").with_inputs("sentence"),
]

val = [
    dspy.Example(sentence="I'm so excited!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="I hate Mondays.", emotion="sad").with_inputs("sentence"),
    dspy.Example(sentence="The sky is blue.", emotion="neutral").with_inputs("sentence"),
]

In [6]:
class Emotion(dspy.Signature):
    """Classify the emotion of a sentence."""
    sentence = dspy.InputField()
    emotion = dspy.OutputField(desc="happy, sad, or neutral")

# Module
class EmotionClassifier(dspy.Module):
    def __init__(self):
        self.program = dspy.ChainOfThought("sentence -> emotion")
        # self.program = dspy.ChainOfThought(Emotion)
    
    def forward(self, sentence):
        return self.program(sentence=sentence)

# Metric with feedback
def metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    correct = example.emotion.lower() == pred.emotion.lower()
    score = 1.0 if correct else 0.0
    
    if pred_name is None:
        return score
    
    if correct:
        feedback = f"Correct! You classified '{example.sentence}' as '{pred.emotion}', which matches the gold label '{example.emotion}'."
    else:
        feedback = f"Incorrect. You classified '{example.sentence}' as '{pred.emotion}', but the correct emotion is '{example.emotion}'. Think about the emotional tone more carefully."
    
    return dspy.Prediction(score=score, feedback=feedback)

In [7]:
from package.base import DriverLM, ModelResponse, Usage
import httpx

ollama_client = httpx.Client(timeout=600.0)

def ollama_request_fn(prompt: str | None = None, messages: list[dict] | None = None, temperature: float = 0.0, max_tokens: int = 256) -> dict:
    if messages is None:
        messages = [{"role": "user", "content": prompt}]
    
    response = ollama_client.post(
        'http://localhost:11434/api/chat',
        json={
            "model": "llama3.2-vision:11b",
            "messages": messages,
            "stream": False,
            "options": {"temperature": temperature}
        }
    )
    response.raise_for_status()
    return response.json()  # Return full Ollama response

def ollama_output_fn(response: dict) -> ModelResponse:
    content = response.get("message", {}).get("content", "")
    model = response.get("model", "custom")
    
    usage = Usage(
        prompt_tokens=response.get("prompt_eval_count", 0),
        completion_tokens=response.get("eval_count", 0),
        total_tokens=response.get("prompt_eval_count", 0) + response.get("eval_count", 0)
    )
    
    return ModelResponse.from_text(text=content.strip(), usage=usage, model=model)


custom_lm = DriverLM(
    request_fn=ollama_request_fn,
    output_fn=ollama_output_fn,
    cache=True
)

custom_lm.clear_cache()  # Clear old cache entries

reflection_lm = DriverLM(
    request_fn=ollama_request_fn,
    output_fn=ollama_output_fn,
    cache=True,
    temperature=1.0
)

reflection_lm.clear_cache()

In [8]:
dspy.configure(lm=custom_lm)

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",
    num_threads=4,
    track_stats=True,
    reflection_lm=reflection_lm
)

optimized = optimizer.compile(
    EmotionClassifier(),
    trainset=train,
    valset=val
)

2026/01/18 21:11:18 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 392 metric calls of the program. This amounts to 65.33 full evals on the train+val set.
2026/01/18 21:11:18 INFO dspy.teleprompt.gepa.gepa: Using 3 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/392 [00:00<?, ?rollouts/s]2026/01/18 21:11:20 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)
2026/01/18 21:11:20 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.3333333333333333 over 3 / 3 examples
GEPA Optimization:   1%|          | 3/392 [00:01<04:04,  1.59rollouts/s]2026/01/18 21:11:20 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.3

Average Metric: 1.00 / 3 (33.3%): 100%|██████████| 3/3 [00:01<00:00,  2.54it/s]

2026/01/18 21:11:21 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)


2026/01/18 21:11:25 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for program.predict: # Instruction for the assistant

## Task Description
Given a sentence, classify the sentiment of the sentence into one of the following emotions: 'happy', 'sad', 'neutral', or 'frustration'.

## Input Format
The input is a string containing a sentence.

## Task Requirements

1. Identify the sentiment of the sentence.
2. Classify the sentiment into one of the following emotions: 'happy', 'sad', 'neutral', or 'frustration'.
3. Provide a reasoning for the classification, which may include analysis of the sentence structure, punctuation, and emotional cues.
4. Ensure the classification is consistent with the gold labels provided.

## Niche and Domain-Specific Information

1. Emotions are not mutually exclusive, and a sentence can express multiple emotions simultaneously.
2. Tone markers such as exclamation marks and question marks can influence the emotional tone of a sentence.
3. Senten

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 374.99it/s]

...

2026/01/18 21:11:50 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 123: All subsample scores perfect. Skipping.
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 123: Reflective mutation did not propose a new candidate
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 124: Selected program 2 score: 1.0



Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 375.02it/s]

2026/01/18 21:11:50 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 124: All subsample scores perfect. Skipping.
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 124: Reflective mutation did not propose a new candidate
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 125: Selected program 2 score: 1.0



Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 333.32it/s]

2026/01/18 21:11:50 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 125: All subsample scores perfect. Skipping.
2026/01/18 21:11:50 INFO dspy.teleprompt.gepa.gepa: Iteration 125: Reflective mutation did not propose a new candidate
GEPA Optimization:  99%|█████████▉| 390/392 [00:32<00:00, 11.91rollouts/s] 

In [9]:
optimized(sentence="I love this!")

Prediction(
    reasoning='The exclamation mark at the end of the sentence indicates a strong positive emotional tone. The word "love" is a strong positive emotional cue, indicating affection and admiration. The overall sentence structure and punctuation suggest a joyful and enthusiastic sentiment.',
    emotion='happy'
)

In [10]:
from pathlib import Path

artifact_dir = Path("./artifacts")
artifact_dir.mkdir(parents=True, exist_ok=True)

# 1. Inspect the optimized prompts
print("=== Optimized Module ===")
for name, predictor in optimized.named_predictors():
    print(f"\n{name}:")
    print(f"Signature: {predictor.signature}")
    if hasattr(predictor, 'demos'):
        print(f"Demos: {len(predictor.demos)} examples")
        for i, demo in enumerate(predictor.demos):
            print(f"  Demo {i+1}: {demo}")

# 2. Save the optimized module
filename= artifact_dir / "instruction_emotion_custom_classifier.json"
optimized.save(filename)
print(f"\nSaved to {filename}")

# 3. Load it later
loaded = EmotionClassifier()
loaded.load(filename)

# 4. Simple inspection
print("\n=== Quick Check ===")
print(f"Module type: {type(optimized)}")
print(f"Has predict: {hasattr(optimized, 'predict')}")

=== Optimized Module ===

program.predict:
Signature: StringSignature(sentence -> reasoning, emotion
    instructions="# Instruction for the assistant\n\n## Task Description\nGiven a sentence, classify the sentiment of the sentence into one of the following emotions: 'happy', 'sad', 'neutral', or 'frustration'. Ensure the classification is consistent with the gold labels provided.\n\n## Input Format\nThe input is a string containing a sentence.\n\n## Task Requirements\n\n1. Identify the sentiment of the sentence.\n2. Classify the sentiment into one of the following emotions: 'happy', 'sad', 'neutral', or 'frustration'.\n3. Provide a reasoning for the classification, which may include analysis of the sentence structure, punctuation, and emotional cues.\n4. Consider the presence of emotional cues such as emotional adverbs and adjectives.\n5. Be aware that emotions are not mutually exclusive, and a sentence can express multiple emotions simultaneously.\n6. Tone markers such as exclamation

In [11]:
loaded(sentence="I love this!")

Prediction(
    reasoning='The exclamation mark at the end of the sentence indicates a strong positive emotional tone. The word "love" is a strong positive emotional cue, indicating affection and admiration. The overall sentence structure and punctuation suggest a joyful and enthusiastic sentiment.',
    emotion='happy'
)

In [12]:
dspy.inspect_history(n=3)





[2026-01-18T21:11:50.981776]

System message:

Your input fields are:
1. `sentence` (str):
Your output fields are:
1. `reasoning` (str): 
2. `emotion` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## sentence ## ]]
{sentence}

[[ ## reasoning ## ]]
{reasoning}

[[ ## emotion ## ]]
{emotion}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        # Instruction for the assistant
        
        ## Task Description
        Given a sentence, classify the sentiment of the sentence into one of the following emotions: 'happy', 'sad', 'neutral', or 'frustration'. Ensure the classification is consistent with the gold labels provided.
        
        ## Input Format
        The input is a string containing a sentence.
        
        ## Task Requirements
        
        1. Identify the sentiment of the sentence.
        2. Classify the sentiment into one of the following emotions: 'happy', 'sad', 'neu